# Crops Prediction

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
import joblib

In [18]:
df=pd.read_csv('yield.csv')
df1=pd.read_csv('rainfall.csv')
df3=pd.read_csv('pesticides.csv')
df4=pd.read_csv('temp.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'yield.csv'

In [ ]:
# EDA of yield
df.rename(columns={'Value':'hg/ha_yield','Area':'area','Year':'year','Item':'item'},inplace=True)
df=df.drop(['Year Code','Element Code','Element','Year Code','Area Code','Domain Code','Domain','Unit','Item Code'],axis=1)
df.shape

(56717, 4)

In [ ]:
# EDA for rainfall
df1.rename(columns={'average_rain_fall_mm_per_year':'average_rainfall',' Area':'area','Year':'year'},inplace=True)
df1['average_rainfall']=pd.to_numeric(df1['average_rainfall'],errors='coerce')
df1=df1.dropna()
df1.shape

(5947, 3)

In [ ]:
df_main=pd.merge(df,df1,on=['year','area'])
df_main.shape

(25385, 5)

In [ ]:
# EDA for Pesticides
df3.rename(index=str,columns={'Area':'area','Year':'year','Value':'presticides_tonnes'},inplace=True)
df3=df3.drop(columns=['Domain','Element','Item','Unit'],axis=1)

In [ ]:
df_main=pd.merge(df_main,df3,on=['area','year'])
df_main.shape

(18949, 6)

In [ ]:
# EDA for Average Temperature
df4.shape
df4=df4.dropna()
df4=df4.rename(index=str,columns={'year':'year','country':'area'})
df4.shape

(68764, 3)

In [ ]:
df_main=pd.merge(df_main,df4,on=['area','year'])
df_main.shape
# df_main.isnull().sum()
# df_main.groupby('item').count()
# df_main['area'].nunique()# Total 101 Nation here
# df_main.groupby(['area'],sort=True)['hg/ha_yield'].sum().nlargest(10)
# df_main.groupby(['item','area'],sort=True)['hg/ha_yield'].sum().nlargest(10)

(28242, 7)

In [ ]:
# corr_data=df_main.select_dtypes(include=['int64','float64']).corr()
# mask=np.zeros_like(corr_data,dtype=bool)
# mask[np.triu_indices_from(mask)]=True
# f,ax=plt.subplots(figsize=(11,9))
# cmap=sns.palette='vlag'
# sns.heatmap(corr_data,mask=mask,cmap=cmap,vmax=.3,center=0,square=True,linewidths=.5,cbar_kws={'shrink':.5})
# plt.show()

In [ ]:
df_final=df_main.drop(columns=['year'],axis=1)
df_final=pd.get_dummies(df_final,columns=['area','item'],prefix=['Country','Item'])
X=df_final.loc[:,df_final.columns!='hg/ha_yield'].values
y=df_final['hg/ha_yield'].values
sc=MinMaxScaler()
X=sc.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
df_final=df_final.drop(['hg/ha_yield'],axis=1)

In [ ]:
pf=PolynomialFeatures(degree=2)
X_new_train=pf.fit_transform(X_train)
X_new_test=pf.fit_transform(X_test)
model=SGDRegressor()
model.fit(X_new_train,y_train)
y_pred_test=model.predict(X_new_test)
y_pred_train=model.predict(X_new_train)
test_score=r2_score(y_test,y_pred_test)
train_score=r2_score(y_train,y_pred_train)
print(train_score)
print(test_score)

0.9486520780402468
0.9438137932935023


In [ ]:
# Albania	Soybeans	1990	7000	1485.0	121.00	16.37
input=['Albania','Soybeans',1485.0,121.00,16.37]
categorical_col=input[:2]
input_df=pd.DataFrame({'average_rainfall':input[2],'presticides_tonnes':input[3],'avg_temp':input[4]},index=[0])
input_df1=df_final.head(1)
input_df1=input_df1.iloc[:,3:]
def update_columns(df, true_columns):
    df[true_columns] = True
    other_columns = df.columns.difference(true_columns)
    df[other_columns] = False
    return df
true_columns = [f'Country_{categorical_col[0]}',f'Item_{categorical_col[1]}']
input_df2= update_columns(input_df1, true_columns)
final_df=pd.concat([input_df,input_df2],axis=1)
test_input=sc.transform(final_df.values)
test_input1=pf.transform(test_input)
z=model.predict(test_input1)
h=(int(((z[0]/100)*2.47105) * 100) / 100)
h

323.58

In [ ]:
df_final.to_csv('test.csv')
df_main.to_csv('main.csv')

In [ ]:
joblib.dump(model,'model.pkl')
joblib.dump(sc,'sc.pkl')
joblib.dump(pf,'pf.pkl')

['pf.pkl']

In [ ]:
df_final

,average_rainfall,presticides_tonnes,avg_temp,Country_Albania,Country_Algeria,Country_Angola,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,...,Item_Cassava,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,1485.0,121.00,16.37,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1485.0,121.00,16.37,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,1485.0,121.00,16.37,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1485.0,121.00,16.37,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,1485.0,121.00,16.37,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28237,657.0,2550.07,19.76,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
28238,657.0,2550.07,19.76,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
28239,657.0,2550.07,19.76,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
28240,657.0,2550.07,19.76,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
